In [1]:
import torch
import torch.nn as nn
from collections import OrderedDict
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

# define datatype
if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
    dtype_long = torch.cuda.LongTensor
else:
    dtype = torch.FloatTensor
    dtype_long = torch.LongTensor

# model formulation
class GNN_LMSC_cell(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, units:int, gcn_type: str,
                    batch_size: int,  # this entry is unnecessary, kept only for backward compatibility
                    width=125, depth=4,):
            super(GNN_LMSC_cell, self).__init__()

            self.in_channels = in_channels
            self.out_channels = out_channels
            self.units = units
            self.gcn_type = gcn_type
            self.batch_size = batch_size  # not needed
            self.depth = depth

            start_dim = units + in_channels
            inside_dim = start_dim
            self.qb =  Quadratic_block(inside_dim, width, depth)
        
            if gcn_type == 'GCNConv':
                self.gconv1 = GCN_block(inside_dim, inside_dim, layer = 1)
                self.gconv2 = GCN_block(inside_dim, inside_dim, layer = 1)
            
            if self.depth>0:
                inside_dim = width
            else:
                inside_dim = in_channels

            self.fc_alpha = nn.Linear(inside_dim,units)
            self.fc_beta = nn.Linear(inside_dim,units)

            for m in self.modules():
                if isinstance(m, nn.Linear):
                    torch.nn.init.xavier_uniform_(m.weight)
                    if m.bias is not None:
                        m.bias.data.fill_(0)
                    # m.bias.data.fill_(0)

    def forward(
        self,
        X: torch.FloatTensor,
        edge_index: torch.LongTensor,
        edge_weight: torch.FloatTensor = None,
        H: torch.FloatTensor = None,
        ) -> torch.FloatTensor:

        # Input strain increment X-> [batch, seq_len, in_dim]
        # Hidden state H -> [batch, node_number, hidden_dim]

        h_t = H
    
        strain_norm = torch.norm(X[:,:,0:6],dim=2)
        x_input = X.clone()
        x_input[:,:,0:6] = (x_input[:,:,0:6].squeeze(1)/(strain_norm + 1e-15)).unsqueeze(1)

        cat_input = torch.cat([x_input.repeat(1,h_t.shape[1],1),h_t], dim = 2)

        # graph embedding
        if self.gcn_type == 'GCNConv':
            G_input1 = self.gconv1(cat_input, edge_index)
            G_input1 = F.relu(G_input1) 
            G_input2 = self.gconv2(cat_input, edge_index)
            G_input2 = F.relu(G_input2) 
        else:
            G_input1 = cat_input
            G_input2 = cat_input

        G_input1 = torch.tanh(self.qb(G_input1))
        G_input2 = torch.tanh(self.qb(G_input2))

        alpha = torch.exp(self.fc_alpha(G_input1))
        beta  = torch.tanh(self.fc_beta(G_input2))

        exp_f =  torch.exp(- alpha * strain_norm.unsqueeze(2).repeat(1,1,self.units))
        h = exp_f * (h_t  - beta) + beta

        return h.squeeze(1)


class GNN_LMSC_Model(nn.Module):
    def __init__(self, args, output_depth=3):
        super(GNN_LMSC_Model, self).__init__()
        self.hidden_dim = args.hidden_dim
        self.seq_len = args.seq_len
        self.output_dim = args.node_output_dim
        self.batch_size = args.batch_size
        self.node_num = args.num_nodes
        self.input_dim = args.node_input_dim
        self.lmsc = GNN_LMSC_cell(args.node_input_dim, args.node_output_dim,
                                   units= args.hidden_dim, gcn_type=args.GCN_type, batch_size=args.batch_size)
        
        if args.out_type == 'FCNN':
            self.decoder = FCNN(args.layers, nn.ReLU)
        else:
            output_depth = args.out_depth
            self.decoder = Quadratic_block(args.hidden_dim, args.node_output_dim, output_depth)

    def forward(self, data):
        '''
        -Data structure
            1. x: input strain increment
            2. edge_index: grain connections in the format of adjacency list
            3. init_ori: initial grain orientations [grain_number, 3] 
        '''
        x, edge_index = data.x.type(dtype), data.edge_index
        edge_index = edge_index.to(device)

        # preprocessing for batch training
        # Input strain increment X-> [batch, seq_len, feature_dim]
        x = x.view(-1,self.seq_len, self.input_dim)
        edge_index = edge_index[:,0:edge_index.size(1)//self.batch_size]

        # Hidden state H -> [batch, node_number, hidden_dim]
        # h0 = torch.zeros(x.size(0), self.node_num, self.hidden_dim)\
        #     .requires_grad_().to(device)
        h0 = torch.zeros(
            x.size(0), self.node_num, self.hidden_dim,
            device=device
        )
        # hidden_out = torch.zeros(x.size(0), self.node_num, x.size(1),\
        #                           self.hidden_dim).requires_grad_().to(device)
        hidden_out = torch.zeros(
            x.size(0), self.node_num, x.size(1), self.hidden_dim,
            device=device
        )
    
        # Assign initial orientation to hidden state
        h0[:,:,0:3] = data.init_ori.view(-1,self.node_num,3).to(device)
        h_last = h0.clone()

        # sequential prediciton
        for i in range(x.size(1)):
            x_input = x[:,i,:].unsqueeze(1).clone()  
            h_last = self.lmsc(x_input, edge_index, H = h_last)
            h_last = h_last.clone()
            hidden_out[:,:,i,:] = h_last.squeeze(1)
        
        # decode the hidden state to Output feature matrix
        # [batch, node_number, hidden_dim] -> [batch, node_number, out_dim]
        hidden_out = self.decoder(hidden_out)
        return hidden_out



class Quadratic_block(nn.Module):
    def __init__(self, input_dim, out_dim, depth):
        super(Quadratic_block, self,).__init__()
        self.modlist1 = nn.ModuleList()
        self.modlist2 = nn.ModuleList()
        self.depth = depth
        for i in range(depth):
            if i == depth-1:
                self.modlist1.append(torch.nn.Linear(input_dim, out_dim, bias=False))
                self.modlist2.append(torch.nn.Linear(input_dim, out_dim, bias=False))
            else:
                self.modlist1.append(torch.nn.Linear(input_dim, input_dim, bias=False))
                self.modlist2.append(torch.nn.Linear(input_dim, input_dim, bias=False))

    def forward(self, x):
        i = 0
        for m in self.modlist1:
            x1 = m(x)
            m2 = self.modlist2[i]
            x2 = m2(x)
            i += 1
            if i < self.depth:
                x1 = torch.tanh(x1)
                x2 = torch.tanh(x2)
            x = x1 * x2
        return x


class GCN_block(nn.Module):
    def __init__(self, in_channels, out_channels, layer,\
                  improved=False, cached=False, add_self_loops=True):
        super(GCN_block, self,).__init__()
        self.modlist = nn.ModuleList()
        self.layer = layer
        for i in range(layer):
            if i == 0:
                self.modlist.append(GCNConv(in_channels, out_channels,\
                                             improved, cached, add_self_loops))
            else:
                self.modlist.append(GCNConv(out_channels, out_channels,\
                                             improved, cached, add_self_loops))

    def forward(self, x, edge_index):
        i = 0
        for m in self.modlist:
            x = m(x, edge_index)
            i += 1
            if i < self.layer:
                x = torch.relu(x)
        return x


class FCNN(torch.nn.Module):
    def __init__(self, layers, activation):
        super(FCNN, self).__init__()
   
        # parameters
        self.depth = len(layers) - 1        
        # set up layer order dict
        self.activation = activation
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out 

c:\Users\Lachlan\anaconda3\envs\gnn_lmsc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Lachlan\anaconda3\envs\gnn_lmsc\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.18.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from types import SimpleNamespace
import torch
from torch_geometric.data import Data

# ----- dummy args -----
args = SimpleNamespace(
    hidden_dim=16,
    seq_len=5,
    node_output_dim=6,
    batch_size=1,
    num_nodes=4,
    node_input_dim=6,
    GCN_type='GCNConv',
    out_type='Quadratic',
    out_depth=2,
    layers=[16, 16, 6]
)

# ----- dummy graph -----
edge_index = torch.tensor([
    [0, 1, 2, 3, 0, 1],
    [1, 0, 3, 2, 2, 3]
], dtype=torch.long)

x = torch.randn(args.seq_len * args.node_input_dim)
init_ori = torch.randn(args.num_nodes, 3)

data = Data(
    x=x,
    edge_index=edge_index,
    init_ori=init_ori
)

# ----- run -----
model = GNN_LMSC_Model(args).to(device)
out = model(data)

print(out.shape)


torch.Size([1, 4, 5, 6])


(8, 101, 100, 15)


In [14]:
import numpy as np
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pickle
import h5py

h5_data = h5py.File('demo_dataset_PN8_sq100.hdf5')
all_data = h5_data['data'][:]

print(all_data.shape)

def nx_to_edge_index(G):
    # Ensure nodes are labeled 0..N-1
    G = nx.convert_node_labels_to_integers(G, ordering="sorted")

    # Get edge list
    edges = list(G.edges())

    # Make edges bidirectional
    row = []
    col = []
    for i, j in edges:
        row += [i, j]
        col += [j, i]

    edge_index = torch.tensor([row, col], dtype=torch.long)
    return edge_index

with open("RVE24-n100_edge_feature.pickle", "rb") as f:
    edges = pickle.load(f)

edge_index_tensor = nx_to_edge_index(edges) # edge index tensor

stress = all_data[:, :100, :, 3:9]   # stress from grains
sigma_min = stress.min()
sigma_max = stress.max()

(8, 101, 100, 15)


In [21]:
def normalize_stress(stress, sigma_min, sigma_max):
    return (2 * (stress - sigma_min) / (sigma_max - sigma_min)) - 1

def denormalize_stress(stress_norm, sigma_min, sigma_max):
    return (stress_norm + 1) * (sigma_max - sigma_min) / 2 + sigma_min

def create_data_object(strain_path): # shape (node_num: grains, seq_len: strain increments, feature_dim: variables)
    """
        Inputs
    """
    init_ori = strain_path[:100, 0, 0:3]     # initial orientation of all 100 grains, excluded 101th grain for homogenized RVE
    init_ori = np.deg2rad(init_ori)    # convert to radians
    init_ori = torch.tensor(init_ori, dtype=torch.float32)

    rve_acc_strain = strain_path[100, :, 9:15]    # accumulated strain  of homogenized RVE. Scaled by 100, following the paper.
    strain_incs = np.zeros((100, 6), dtype=np.float32)
    strain_incs[1:] = np.diff(rve_acc_strain, axis=0)    # calculating strain increment, from accumulated strain. First element remains as 0.
    strain_incs *= 100    # Output multiplied by 100, follows the paper.
    strain_incs = torch.tensor(strain_incs, dtype=torch.float32)

    edge_index = edge_index_tensor

    data = Data(
        x=strain_incs,       # shape (seq_len, 6)
        edge_index=edge_index,
        init_ori=init_ori    # shape (100,3)
    )
    return data


def create_ground_truth(strain_path, sigma_min, sigma_max):
    """
    Returns normalized ground truth
    """
    y = np.copy(strain_path[:100, :, :])        # (100, 100, 15), omit 101th grain (RVE homogenized)

    y[:, :, 0:3] = np.deg2rad(y[:, :, 0:3]) # convert orientation to radians

    # normalize stress channels only (3:9)
    y_stress = y[:, :, 3:9]
    y[:, :, 3:9] = normalize_stress(y_stress, sigma_min, sigma_max)

    y = torch.tensor(y, dtype=torch.float32)
    return y

In [22]:
import torch
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader

train_data = all_data[:6]   # first 6 paths
test_data  = all_data[6:]   # last 2 paths


class StrainPathDataset(Dataset):
    def __init__(self, data, sigma_min, sigma_max):
        self.data = data    # loaded strain path from hdf5 demo dataset (paths, 101, 100, 15)
        self.sigma_min = sigma_min
        self.sigma_max = sigma_max

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        strain_path = self.data[idx]  # (101, 100, 15)

        data = create_data_object(strain_path)
        y = create_ground_truth(strain_path, self.sigma_min, self.sigma_max)

        return data, y

train_dataset = StrainPathDataset(train_data, sigma_min, sigma_max)
test_dataset  = StrainPathDataset(test_data,  sigma_min, sigma_max)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=1, shuffle=False)

Define model

In [23]:
from argparse import Namespace

args = Namespace(
    hidden_dim=64,
    seq_len=100,
    node_output_dim=15,     
    batch_size=2,
    num_nodes=100,
    node_input_dim=6,
    GCN_type='GCNConv',
    out_type='Quadratic',   # NOT 'FCNN'
    out_depth=3,
    layers=None             # unused since out_type != 'FCNN'
)

model = GNN_LMSC_Model(args)
model = model.to(device)


In [27]:
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.MSELoss()

num_epochs = 1000

for epoch in range(num_epochs):
    epoch_loss = 0.0

    for data, y_true in train_loader:
        data = data.to(device)
        y_true = y_true.to(device)

        optimizer.zero_grad()

        y_pred = model(data)
        # loss = criterion(y_pred, y_true)
        loss = torch.mean((y_pred - y_true)**2)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    print(f"Epoch {epoch} | Loss: {epoch_loss:.6f}")


Epoch 0 | Loss: 0.075757
Epoch 1 | Loss: 0.073933
Epoch 2 | Loss: 0.072669
Epoch 3 | Loss: 0.071637
Epoch 4 | Loss: 0.070752
Epoch 5 | Loss: 0.069801
Epoch 6 | Loss: 0.068816
Epoch 7 | Loss: 0.068030
Epoch 8 | Loss: 0.067131
Epoch 9 | Loss: 0.066315
Epoch 10 | Loss: 0.065637
Epoch 11 | Loss: 0.064961
Epoch 12 | Loss: 0.064274
Epoch 13 | Loss: 0.063737
Epoch 14 | Loss: 0.062985
Epoch 15 | Loss: 0.062289
Epoch 16 | Loss: 0.061702
Epoch 17 | Loss: 0.061030
Epoch 18 | Loss: 0.060300
Epoch 19 | Loss: 0.059786
Epoch 20 | Loss: 0.059020
Epoch 21 | Loss: 0.058273
Epoch 22 | Loss: 0.057664
Epoch 23 | Loss: 0.057319
Epoch 24 | Loss: 0.056899
Epoch 25 | Loss: 0.056432
Epoch 26 | Loss: 0.056067
Epoch 27 | Loss: 0.055688
Epoch 28 | Loss: 0.055284
Epoch 29 | Loss: 0.055004
Epoch 30 | Loss: 0.054707
Epoch 31 | Loss: 0.054462
Epoch 32 | Loss: 0.054281
Epoch 33 | Loss: 0.054017
Epoch 34 | Loss: 0.053967
Epoch 35 | Loss: 0.053588
Epoch 36 | Loss: 0.053307
Epoch 37 | Loss: 0.053018
Epoch 38 | Loss: 0.052

KeyboardInterrupt: 

In [28]:
import torch.nn.functional as F

model.eval()

all_preds = []
all_trues = []

with torch.no_grad():
    for data, y_true in test_loader:
        data = data.to(device)
        y_true = y_true.to(device)

        y_pred = model(data)

        all_preds.append(y_pred.cpu())
        all_trues.append(y_true.cpu())

ori_mse = F.mse_loss(y_pred[..., 0:3], y_true[..., 0:3])
stress_mse = F.mse_loss(y_pred[..., 3:9], y_true[..., 3:9])
acc_strain_mse = F.mse_loss(y_pred[..., 9:15], y_true[..., 9:15])

print("Orientation MSE:", ori_mse.item())
print("Stress MSE:", stress_mse.item())
print("Accumulated strain MSE:", acc_strain_mse.item())

Orientation MSE: 0.18127737939357758
Stress MSE: 0.10055667161941528
Accumulated strain MSE: 0.006004763767123222


In [31]:
b = 0         # batch index
g = 31       # grain index
t = 50         # timestep


print("Prediction orientation: ", y_pred[b, g, t, 0:3])
print("Ground truth orientation: ", y_true[b, g, t, 0:3])
print('Original truth orientation: ', all_data[b+6, g, t, 0:3])
print('\n')

print("Prediction Cauchy stress: ", y_pred[b, g, t, 3:9])
print("Ground truth Cauchy stress: ", y_true[b, g, t, 3:9])
print("Original truth Cauchy stress: ", all_data[b+6, g, t, 3:9])
print('\n')

print("Prediction accumulated strain: ", y_pred[b, g, t, 9:15])
print("Ground truth accumulated strain: ", y_true[b, g, t, 9:15])
print('Original truth accumulated strain: ', all_data[b+6, g, t, 9:15])
print('\n')

Prediction orientation:  tensor([0.2828, 0.2823, 0.4492], device='cuda:0')
Ground truth orientation:  tensor([3.8973, 0.3591, 1.0479], device='cuda:0')
Original truth orientation:  [221.54156  20.53923  59.7123 ]


Prediction Cauchy stress:  tensor([-0.0920,  0.0056, -0.0060,  0.0030, -0.0114,  0.0235], device='cuda:0')
Ground truth Cauchy stress:  tensor([ 0.5557, -0.0793,  0.1501,  0.0526,  0.0274, -0.0987], device='cuda:0')
Original truth Cauchy stress:  [232.73143 -39.71717  60.34594  19.38458   7.66089 -44.4092 ]


Prediction accumulated strain:  tensor([-0.0201,  0.0246,  0.0153, -0.0002,  0.0009, -0.0051], device='cuda:0')
Ground truth accumulated strain:  tensor([ 0.0896, -0.0267, -0.0496, -0.0158, -0.0099,  0.0013], device='cuda:0')
Original truth accumulated strain:  [ 0.08904 -0.0275  -0.04836 -0.01587 -0.00992  0.0015 ]




In [ ]:
print()

In [ ]:
import h5py

h5_data = h5py.File('demo_dataset_PN8_sq100.hdf5')
all_data = h5_data['data'][:]
# print(all_data.shape)

test_data = all_data[1]

# orig_ori = test_data[g, t, ]
print(test_data[g, t, ])

# test = all_data[:100, :, :]        # (100, 100, 15)
test = all_data[7, g, t, 3:9]        # (100, 100, 15)

# normalize stress channels only (3:9)
# test_stress = test[:, :, 3:9]
normalised_stress = normalize_stress(test, sigma_min, sigma_max)

# print(normalised_stress)
# print(sigma_min, sigma_max)

processed_data = create_data_object(test_data)
# print(processed_data.x)

[-390.3163  -110.51343   82.76505   -2.18077   44.80113  -60.64523]


In [48]:
from torch import device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data, y_true = next(iter(loader))
data = data.to(device)
y_true = y_true.to(device)

y_pred = model(data)

print(y_pred.shape)
# print(data.x.shape)
print(y_true.shape)


torch.Size([2, 100, 100, 15])
torch.Size([2, 100, 100, 15])


In [ ]:
import pickle
import networkx as nx
import torch

with open("RVE24-n100_edge_feature.pickle", "rb") as f:
    edges = pickle.load(f)


def nx_to_edge_index(G):
    # Ensure nodes are labeled 0..N-1
    G = nx.convert_node_labels_to_integers(G, ordering="sorted")

    # Get edge list
    edges = list(G.edges())

    # Make edges bidirectional
    row = []
    col = []
    for i, j in edges:
        row += [i, j]
        col += [j, i]

    edge_index = torch.tensor([row, col], dtype=torch.long)
    return edge_index


torch.Size([2, 1504])
0 2
2 0
0 12
12 0
0 13
13 0
0 17
17 0
0 30
30 0
0 34
34 0
0 40
40 0
0 46
46 0
0 50
50 0
0 57
57 0
0 65
65 0
0 68
68 0
0 72
72 0
0 81
81 0
0 89
89 0
0 92
92 0
0 97
97 0
2 4
4 2
2 17
17 2
2 18
18 2
2 34
34 2
2 36
36 2
2 40
40 2
2 59
59 2
2 60
60 2
2 68
68 2
2 74
74 2
2 79
79 2
2 82
82 2
2 85
85 2
2 89
89 2
2 98
98 2
12 13
13 12
12 15
15 12
12 30
30 12
12 31
31 12
12 37
37 12
12 38
38 12
12 57
57 12
12 61
61 12
12 72
72 12
12 80
80 12
12 91
91 12
12 95
95 12
12 97
97 12
13 5
5 13
13 37
37 13
13 38
38 13
13 46
46 13
13 57
57 13
13 58
58 13
13 68
68 13
13 81
81 13
13 85
85 13
13 92
92 13
13 97
97 13
13 99
99 13
17 18
18 17
17 24
24 17
17 34
34 17
17 48
48 17
17 50
50 17
17 65
65 17
17 68
68 17
17 74
74 17
17 82
82 17
30 11
11 30
30 26
26 30
30 31
31 30
30 39
39 30
30 50
50 30
30 57
57 30
30 61
61 30
30 72
72 30
30 76
76 30
30 78
78 30
30 91
91 30
30 98
98 30
34 4
4 34
34 32
32 34
34 37
37 34
34 40
40 34
34 50
50 34
34 56
56 34
34 59
59 34
34 60
60 34
34 61
61 34
34 72


In [104]:
# data_obj = create_data_object(data[0])

# print(data_obj.x.shape)

# for batch in loader:
#     print("x:", batch.x.shape)
#     print("edge_index:", batch.edge_index.shape)
#     print("init_ori:", batch.init_ori.shape)
#     print("batch vector:", batch.batch.shape)
#     break

print(path0[0, :5, 0:3])
print(np.deg2rad(path0[0, :5, 0:3]))              # convert to radians

[[228.0067   18.6711   15.5926 ]
 [227.94653  18.63262  15.63924]
 [227.88055  18.59222  15.68936]
 [227.80481  18.55232  15.74941]
 [227.7251   18.51262  15.81334]]
[[3.9794676  0.32587218 0.2721422 ]
 [3.9784174  0.32520056 0.27295622]
 [3.9772658  0.32449546 0.27383098]
 [3.975944   0.32379907 0.27487904]
 [3.9745526  0.3231062  0.27599484]]


In [98]:
import numpy as np

# for path in data:
#     print(path[13, 0, 9:15])

#     # print(len(path[0, :, 9:15]))

# print(data[0, :, 0, 9:15])

path0 = data[0]

print(path0[:100, 0, 9:15])
# print(np.diff(path0[100, :, 9:15][:3], axis=0))


test = [[1, 1, 1], [2, 2, 2], [3, 3, 3], [4, 4, 4]]
# print(np.diff(test))

arr = np.array([[1, 2],
                [2, 3]])

# print(arr.shape)  # (2, 2)

# print(np.diff(arr, axis=0))

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0.

In [26]:
edge_index = torch.tensor([
    [i for i in range(99) for _ in (0,)],
    [i+1 for i in range(99)]
], dtype=torch.long)

print(edge_index)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
         72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
         90, 91, 92, 93, 94, 95, 96, 97, 98],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
         55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
         73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90,
         91, 92, 93, 94, 95, 96, 97, 98, 99]])
